# Zero-Coupon Bond Using Marlowe Runtime\'s REST Interface

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

In this demonsration we use Marlowe Runtime\'s REST interface, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Lender")
            (Role "Lender")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Role "Lender")
            (Party (Role "Borrower"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Role "Borrower")
                        (Role "Borrower")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Role "Borrower")
                        (Party (Role "Lender"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close
```

## Connecting to Marlowe Runtime

If you are executing this example within [demeter.run](https://demeter.run/), then all of the required environment variables have been set to use Marlowe Runtime on Cardano\'s pre-production network.

If not, the following environment variables must be set to point to the Marlowe Runtime deployment:
- `CARDANO_NODE_SOCKET_PATH`
- `CARDANO_TESTNET_MAGIC`
- `MARLOWE_RT_TX_HOST`
- `MARLOWE_RT_TX_COMMAND_PORT`
- `MARLOWE_RT_HISTORY_HOST`
- `MARLOWE_RT_HISTORY_SYNC_PORT`
- `MARLOWE_RUNTIME_URL`

Furthermore, the following executables must be on the `PATH`:
- `marlowe` and `marlowe-cli`` from [github:input-output-hk/marlowe-cardano](https://github.com/input-output-hk/marlowe-cardano/)
- `cardano-cli` from [github:input-output-hk/cardano-node](https://github.com/input-output-hk/cardano-node/)
- `curl`
- `jq`
- `sed`

## Preliminaries

Set up a few things before working with the contract.

### Constants

Set a few convenient constants for later use.

One ada is one million lovelace, which is the basic unit of currency.

In [1]:
ADA=1000000

In Marlowe, time is measured in milliseconds.

In [2]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

### Create keys and addresses for the parties to the contract

A lender and borrower will interact with the contract. In order to submit transactions, each of them needs a wallet with an address and a signing key.

#### The lender

Set the file names for this party's signing key and verification key.

In [3]:
LENDER_SKEY=lender.skey
LENDER_VKEY=lender.vkey

Generate the keys if they haven't already been generated.

In [4]:
if [[ ! -e "$LENDER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$LENDER_SKEY" \
    --verification-key-file "$LENDER_VKEY"
fi

Compute the party's address on the testnet.

In [5]:
LENDER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$LENDER_VKEY" )
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqer2uqv8raxqvad4vyt6mczad69ms5atuja4rwfck0mp8s9mcpxc


#### The borrower

Set the file names for this party's signing key and verification key.

In [6]:
BORROWER_SKEY=borrower.skey
BORROWER_VKEY=borrower.vkey

Generate the keys if they haven't already been generated.

In [7]:
if [[ ! -e "$BORROWER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$BORROWER_SKEY"  \
    --verification-key-file "$BORROWER_VKEY"
fi

Compute the party's address on the testnet.

In [8]:
BORROWER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BORROWER_VKEY" )
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vr0xg68f7grs68cjd4ejgar7ymvyf55c9q7cyknjxufhadcmn03sc


### Fund the parties' wallets

If the parties do not already have funds in their wallets, send test ada to them. Here are the faucets for the public testnets: https://docs.cardano.org/cardano-testnet/tools/faucet.

It is most convenient to send the funds to Daedalus, and then use Daedalus to fund the three parties. You can download testnet versions of Daedalus at https://docs.cardano.org/cardano-testnet/daedalus-testnet.

However, for this example, we'll use a private faucet to send funds to the three addresses.

In [9]:
marlowe-cli util fund-address \
 --lovelace "$((100 * ADA))" \
 --out-file /dev/null \
 --source-wallet-credentials "$(cat ../faucet.testnet.address):../faucet.skey" \
 --submit 600 \
 "$LENDER_ADDR" "$BORROWER_ADDR"

TxId "a798acad7e780498a3207fa80f75a1141d15e1347508ba4eb6f476c5fb1de638"


See that the addresses have indeed been funded:

In [10]:
echo
echo "Lender @ $LENDER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo

echo
echo "Borrower @ $BORROWER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo


Lender @ addr_test1vqer2uqv8raxqvad4vyt6mczad69ms5atuja4rwfck0mp8s9mcpxc
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a798acad7e780498a3207fa80f75a1141d15e1347508ba4eb6f476c5fb1de638     1        100000000 lovelace + TxOutDatumNone


Borrower @ addr_test1vr0xg68f7grs68cjd4ejgar7ymvyf55c9q7cyknjxufhadcmn03sc
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a798acad7e780498a3207fa80f75a1141d15e1347508ba4eb6f476c5fb1de638     2        100000000 lovelace + TxOutDatumNone



## Design the Contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Here we generate the contract using Marlowe CLI.

First find the current time.

In [11]:
NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $NOW POSIX milliseconds = $(date -d @$((NOW / SECOND)))"

NOW = 1676932293000 POSIX milliseconds = Mon Feb 20 03:31:33 PM MST 2023


Set the principal to 80 ada and the interest to 5 ada.

In [12]:
PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))
echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [13]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo "LENDER_DEADLINE = $LENDER_DEADLINE POSIX milliseconds = $(date -d @$((LENDER_DEADLINE / SECOND)))"
echo "BORROWER_DEADLINE = $BORROWER_DEADLINE POSIX milliseconds = $(date -d @$((BORROWER_DEADLINE / SECOND)))"

LENDER_DEADLINE = 1676935893000 POSIX milliseconds = Mon Feb 20 04:31:33 PM MST 2023
BORROWER_DEADLINE = 1676943093000 POSIX milliseconds = Mon Feb 20 06:31:33 PM MST 2023


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [14]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Now create the JSON file for the contract.

In [15]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender Lender \
  --borrower Borrower \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file /dev/null

View the contract file as YAML.

In [16]:
json2yaml zcb-contract.json

timeout: 1676935893000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  then:
    from_account:
      role_token: Lender
    pay: 80000000
    then:
      timeout: 1676943093000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            role_token: Borrower
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: Borrower
        then:
          from_account:
            role_token: Borrower
          pay:
            add: 80000000
            and: 5000000
          then: close
          to:
            party:
              role_token: Lender
          token:
            currency_symbol: ''
            token_name: ''
    to:
      party:
        role_token: Borro

## How to check the safety of a Marlowe contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/)
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

## Transaction 1. Create the Contract

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [17]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat zcb-contract.json)
roles:
  Lender: $LENDER_ADDR
  Borrower: $BORROWER_ADDR
minUTxODeposit: $MIN_LOVELACE
metadata: {}
EOI
cat request-1.json

{"contract":{"timeout":1676935893000,"timeout_continuation":"close","when":[{"case":{"deposits":80000000,"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Lender"}},"then":{"from_account":{"role_token":"Lender"},"pay":80000000,"then":{"timeout":1676943093000,"timeout_continuation":"close","when":[{"case":{"deposits":{"add":80000000,"and":5000000},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Borrower"}},"then":{"from_account":{"role_token":"Borrower"},"pay":{"add":80000000,"and":5000000},"then":"close","to":{"party":{"role_token":"Lender"}},"token":{"currency_symbol":"","token_name":""}}}]},"to":{"party":{"role_token":"Borrower"}},"token":{"currency_symbol":"","token_name":""}}}]},"metadata":{},"minUTxODeposit":2000000,"roles":{"Borrower":"addr_test1vr0xg68f7grs68cjd4ejgar7ymvyf55c9q7cyknjxufhadcmn03sc","Lender":"addr_test1vqer2uqv8raxqvad4vyt6mczad69ms5atuj

Next we post the request and view the response.

In [18]:
curl '$MARLOWE_RUNTIME_URL/contracts' \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -s
json2yaml response-1.json

links:
  contract: contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231
resource:
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  txBody:
    cborHex: 86a80081825820a798acad7e780498a3207fa80f75a1141d15e1347508ba4eb6f476c5fb1de638010d81825820a798acad7e780498a3207fa80f75a1141d15e1347508ba4eb6f476c5fb1de638010184a200581d603235700c38fa6033adab08bd6f02eb745dc29d5f25da8dc9c59fb09e011a05b0dbdaa300581d706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0011a001e84800282005820f545fb31f4fc72c6f69e73efb3262251d1e378ead2519d73f94cf8b8dcb2b156a200581d60de6468e9f2070d1f126d7324747e26d844d298283d825a7237137eb701821a000fea4ca1581c4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410cea148426f72726f77657201a200581d603235700c38fa6033adab08bd6f02eb745dc29d5f25da8dc9c59fb09e01821a000fc8a0a1581c4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410cea1464c656e6465720110a200581d603235700c38fa6033adab08bd6f02eb745dc29d5f25da8dc9c59fb09e011a05

The identifier for the contract is embedded in the response.

In [19]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1


The CBOR serialization (in text-envelope format) is also embedded in the response.

In [20]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [21]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285


One can view the transaction on a Cardano explorer.

In [22]:
echo "https://preprod.cardanoscan.io/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285?tab=utxo


In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [23]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "f545fb31f4fc72c6f69e73efb3262251d1e378ead2519d73f94cf8b8dcb2b156"


## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [24]:
CONTRACT_URL="$MARLOWE_RUNTIME_URL/contracts/$(jq -r '.links.contract' response-1.json)"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://localhost:8089/contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231


In [25]:
curl -s "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231/transactions
resource:
  block:
    blockHeaderHash: 235608616b6a5e29e6648329d05a2450a8c018c6386c4895bebde1dd43bf1ea8
    blockNo: 651014
    slotNo: 21249128
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  currentContract:
    timeout: 1676935893000
    timeout_continuation: close
    when:
    - case:
        deposits: 80000000
        into_account:
          role_token: Lender
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Lender
      then:
        from_account:
          role_token: Lender
        pay: 80000000
        then:
          timeout: 1676943093000
          timeout_continuation: close
          when:
          - case:
              deposits:
                add: 80000000
                and: 5000000
              into_account:
                role_token: Borrower
           

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The lender is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [26]:
marlowe-cli input deposit \
  --deposit-party Lender \
  --deposit-account Lender \
  --deposit-amount "$PRINCIPAL" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Lender
into_account:
  role_token: Lender
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 80000000


This input is included in the JSON request.

In [27]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Lender"},"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":80000000}],"metadata":{},"version":"v1"}


Next we post the request and store the response.

In [28]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -s
json2yaml response-2.json

links:
  transaction: contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231/transactions/edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910
resource:
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  transactionId: edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910
  txBody:
    cborHex: 86aa0083825820f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f28500825820f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f28501825820f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285030d81825820f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285001281825820d1e488d70850fc36ef6971a65d1f45e2a7a433e4080e4c5c580d0b23094a955e010184a200581d603235700c38fa6033adab08bd6f02eb745dc29d5f25da8dc9c59fb09e011a00dfec2ba300581d706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0011a001e84800282005820649c5cce0b8fecc93f7011bc7e3ffeb99ab15fc6f0b658100c57d8c6a8fe1399a200581d6032357

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [29]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [30]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910


One can view the transaction on a Cardano explorer.

In [31]:
echo "https://preprod.cardanoscan.io/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910?tab=utxo


One can see that the lender has 16 ada instead of 100 ada. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction. The other ada was used for fees. The lender also holds their own role token.

In [32]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910     0        14674987 lovelace + TxOutDatumNone
edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910     2        1034400 lovelace + 1 4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410ce.4c656e646572 + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation.

In [33]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "649c5cce0b8fecc93f7011bc7e3ffeb99ab15fc6f0b658100c57d8c6a8fe1399"


Marlowe\'s role-payout address holds the 80 ada on behalf of the borrower.

In [34]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910     3        80000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "dff6648987ec6a6472a6183a76ee9aea813d81923b7a467e2f1fe9a300b629cb"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [35]:
curl -s "$CONTRACT_URL/transactions/$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: 50a9528afabbbb9e65769022ea8d7ddf5f9082e9e190e9bfcf5c8084b62fcc38
    blockNo: 651018
    slotNo: 21249200
  consumingTx: null
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  inputUtxo: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  inputs:
  - input_from_party:
      role_token: Lender
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 80000000
  invalidBefore: '2023-02-20T22:32:26Z'
  invalidHereafter: '2023-02-20T23:31:33Z'
  metadata: {}
  outputContract:
    timeout: 1676943093000
    timeout_continuation: close
    when:
    - case:
        deposits:
          add: 80000000
          and: 5000000
        into_account:
          role_token: Borrower
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Borrower
      then:
        from_account:
      

## Transaction 3. The borrower withdraws the principal

Sadly, there currently is no Marlowe Runtime REST endpoint for withdrawing funds from the role-payout address. Instead, we use Marlowe Runtime\'s command-line interface, `marlowe`, to do this.

In [36]:
TX_3=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role Borrower \
  --change-address "$BORROWER_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

TX_3 = e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998


Submit the transaction.

In [37]:
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600

TxId "e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998"


On can view the transaction on a Cardano explorer.

In [38]:
echo "https://preprod.cardanoscan.io/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998?tab=utxo


The borrower now has about 180 ada. The borrower also holds their own role token.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998     0        179664371 lovelace + TxOutDatumNone
e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998     1        1043020 lovelace + 1 4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410ce.426f72726f776572 + TxOutDatumNone


## Transaction 4. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

First build the input to deposit the funds to the contract.

In [40]:
marlowe-cli input deposit \
  --deposit-party Borrower \
  --deposit-account Borrower \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --out-file input-4.json
json2yaml input-4.json

input_from_party:
  role_token: Borrower
into_account:
  role_token: Borrower
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 85000000


Next build the request.

In [41]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
EOI
cat request-4.json

{"inputs":[{"input_from_party":{"role_token":"Borrower"},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":85000000}],"metadata":{},"version":"v1"}


Now post the request to Marlowe Runtime.

In [42]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -s
json2yaml response-4.json

links:
  transaction: contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231/transactions/982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff
resource:
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  transactionId: 982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff
  txBody:
    cborHex: 86aa0083825820e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed199800825820e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed199801825820edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910010d81825820e0b43f101dfa388cd6288ccbd6282bc91bbf6a9081d1c065a403e13174ed1998001281825820d1e488d70850fc36ef6971a65d1f45e2a7a433e4080e4c5c580d0b23094a955e010184a200581d60de6468e9f2070d1f126d7324747e26d844d298283d825a7237137eb7011a059a0330a200581d60de6468e9f2070d1f126d7324747e26d844d298283d825a7237137eb701821a000fea4ca1581c4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410cea148426f72726f77657201a30

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [43]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [44]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff


One can view the transaction on a Cardano explorer.

In [45]:
echo "https://preprod.cardanoscan.io/transaction/$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff?tab=utxo


The borrower now has a total of about 95 ada, which is 100 ada minus the 5 ada paid as interest.

In [46]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff     0        93979440 lovelace + TxOutDatumNone
982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff     1        1043020 lovelace + 1 4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410ce.426f72726f776572 + TxOutDatumNone


The 85 ada are at Marlowe\'s role-payout address, held on behalf of the lender.

In [47]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff     2        85000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "4a74637557de6ec17d0e2886d5aa5d52fa5224918fa9048c00f0768b37568297"


The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [48]:
curl -s "$CONTRACT_URL/transactions/$TX_4" | json2yaml

links:
  previous: contracts/f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285%231/transactions/edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910
resource:
  block:
    blockHeaderHash: c3fc9aa3c00da8a30886c208d6b1c33ab349bc3e9a59dd98beb6558f645d70cd
    blockNo: 651021
    slotNo: 21249299
  consumingTx: null
  contractId: f5a19b1f679b926d4fca1a240584595ba1d0da74cf1c29d0ad3c13c2b920f285#1
  inputUtxo: edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910#1
  inputs:
  - input_from_party:
      role_token: Borrower
    into_account:
      role_token: Borrower
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 85000000
  invalidBefore: '2023-02-20T22:33:55Z'
  invalidHereafter: '2023-02-21T01:31:33Z'
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  transactionId: 982cbc1fd3fc6f89eaf2a3c0368e97d61f91ca336abe60fbd83fe7919b22b3ff
  txBody: null


## Transaction 5. The lender withdraws their principal and interest

Here again we use `marlowe` to withdraw funds from the Marlowe role-payout address.

In [49]:
TX_5=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role Lender \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-5.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_5 = $TX_5"

TX_5 = 380f2c92fb064c60f04f132cc894310b2f647b2f78c6adf53971abd65ea38872


Submit the transaction.

In [50]:
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600

TxId "380f2c92fb064c60f04f132cc894310b2f647b2f78c6adf53971abd65ea38872"


One can view the transaction on a Cardano explorer.

In [51]:
echo "https://preprod.cardanoscan.io/transaction/$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/380f2c92fb064c60f04f132cc894310b2f647b2f78c6adf53971abd65ea38872?tab=utxo


The lender now has a little over 102 ada, having received the interest and having paid transaction fees.

In [52]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
380f2c92fb064c60f04f132cc894310b2f647b2f78c6adf53971abd65ea38872     0        86664547 lovelace + TxOutDatumNone
380f2c92fb064c60f04f132cc894310b2f647b2f78c6adf53971abd65ea38872     1        1034400 lovelace + 1 4923a4105d5f5fbad2fb0ae63b2c68a7fe0eae2201c9c9a09ed410ce.4c656e646572 + TxOutDatumNone
edc3c900b7d85632dbf96eb99a8bc5d2a9389d48155e494d0e21acb2c2664910     0        14674987 lovelace + TxOutDatumNone


## \[Optional\] Clean Up

We can tidy up the wallets by sending their funds back to the original faucet.

In [53]:
POLICY_ID=$(curl -s $CONTRACT_URL | jq -r '.resource.roleTokenMintingPolicyId')
marlowe-cli transaction simple \
  --tx-in "$TX_2#0" \
  --tx-in "$TX_4#0" \
  --tx-in "$TX_4#1" \
  --tx-in "$TX_5#0" \
  --tx-in "$TX_5#1" \
  --tx-out "addr_test1vqxdw4rlu6krp9fwgwcnld6y84wdahg585vrdy67n5urp9qyts0y7+1250000+1 $POLICY_ID.Lender+1 $POLICY_ID.Borrower" \
  --change-address "$(cat ../faucet.testnet.address)" \
  --required-signer "$LENDER_SKEY" \
  --required-signer "$BORROWER_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "e5cc68288b6b5fdde55231694ab7d5db4b1747e1cc4b488abb073342161c6dac"


## *Appendix:* OpenApi Description of Marlowe Web Services

Here is the OpenApi description of endpoints and types provided by `marlowe-web-server`.

In [54]:
curl -s '$RUNTIME_URL/openapi.json' | json2yaml

components:
  schemas:
    Action:
      description: A contract which becomes active when an action occurs.
      oneOf:
      - properties:
          deposits:
            $ref: '#/components/schemas/Value'
          into_account:
            $ref: '#/components/schemas/Party'
          of_token:
            $ref: '#/components/schemas/Token'
          party:
            $ref: '#/components/schemas/Party'
        required:
        - party
        - deposits
        - of_token
        - into_account
        type: object
      - properties:
          choose_between:
            items:
              $ref: '#/components/schemas/Bound'
            type: array
          for_choice:
            $ref: '#/components/schemas/ChoiceId'
        required:
        - for_choice
        - choose_between
        type: object
      - properties:
          notify_if:
            $ref: '#/components/schemas/Observation'
        required:
        - notify_if
        type: object
    Address:
      descri